# Converting a group of csv files with our data to a SQLite database

The purpose of the following notebook is to highlight the process of turning the csv files into a usable, and easily accessible database

In [1]:
#import dependencies
import pandas as pd
import sqlite3
import sqlalchemy as sql
import datetime as dt

##### Set Up database ######
engine = sql.create_engine("sqlite:///../Database/sales.db", echo=True)


In [2]:
#load the file(s) we need into a pandas df
path = "../Resources"
weather_df = pd.read_csv(f"{path}/week_weather_summary.csv")
sales_df = pd.read_csv(f"{path}/weekly_sales_complete_Type.csv")

In [3]:
#preview both dfs

## Weather

In [4]:
weather_df.head()

,dt,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,clouds_all,weather_main_Clear,weather_main_Clouds,weather_main_Drizzle,weather_main_Fog,weather_main_Haze,weather_main_Mist,weather_main_Rain,weather_main_Smoke,weather_main_Snow,weather_main_Thunderstorm
0,2019-02-15,23.368876,14.429101,-5.10,41.14,1018.101124,76.247191,11.688090,50.320225,0.410112,0.224719,0.005618,0.005618,0.000000,0.039326,0.067416,0.0,0.247191,0.0
1,2019-02-22,22.586823,18.859063,0.37,42.76,1019.541667,77.239583,3.882969,61.526042,0.229167,0.213542,0.041667,0.010417,0.062500,0.088542,0.130208,0.0,0.223958,0.0
2,2019-03-01,15.662619,11.220119,-8.72,31.91,1022.636905,64.565476,3.630298,43.059524,0.464286,0.386905,0.000000,0.000000,0.053571,0.005952,0.000000,0.0,0.089286,0.0
3,2019-03-08,34.007813,31.361771,5.77,62.64,1012.838542,79.098958,3.752135,52.005208,0.302083,0.203125,0.057292,0.010417,0.057292,0.109375,0.239583,0.0,0.020833,0.0
4,2019-03-15,36.160057,33.896761,21.74,52.23,1019.306818,71.698864,3.503636,48.096591,0.392045,0.340909,0.028409,0.000000,0.051136,0.079545,0.102273,0.0,0.005682,0.0


In [5]:
#checking data types
weather_df.dtypes

dt                            object
temp                         float64
feels_like                   float64
temp_min                     float64
temp_max                     float64
pressure                     float64
humidity                     float64
wind_speed                   float64
clouds_all                   float64
weather_main_Clear           float64
weather_main_Clouds          float64
weather_main_Drizzle         float64
weather_main_Fog             float64
weather_main_Haze            float64
weather_main_Mist            float64
weather_main_Rain            float64
weather_main_Smoke           float64
weather_main_Snow            float64
weather_main_Thunderstorm    float64
dtype: object

In [6]:
#converting respective date column into datetime format for easier comparison
weather_df['dt'] = pd.to_datetime(weather_df['dt'])

## Sales

In [7]:
sales_df.head()

,item,item_code,quantity,unit_price,total_sales_amount,date,Type
0,PINT Spotted Cow,3140,64.0,3.937500,252.00,2021-01-08,Beer
1,DBL RAIL Vodka,3455,37.0,4.945946,183.00,2021-01-08,Vodka
2,BTL Miller High Life,3122,31.0,3.250000,100.75,2021-01-08,Beer
3,PINT Stein,3141,29.0,3.517241,102.00,2021-01-08,Beer
4,SHOT Seagrams VO,3325,26.0,2.423077,63.00,2021-01-08,Scotch/Whiskey


In [8]:
#checking data types
sales_df.dtypes

item                   object
item_code               int64
quantity              float64
unit_price            float64
total_sales_amount    float64
date                   object
Type                   object
dtype: object

In [9]:
#converting respective date column into datetime format for easier comparison
sales_df['date'] = pd.to_datetime(sales_df['date'])

#converting quantity column to int
sales_df['quantity'] = sales_df['quantity'].astype(int)

In [10]:
sales_df.dtypes

item                          object
item_code                      int64
quantity                       int32
unit_price                   float64
total_sales_amount           float64
date                  datetime64[ns]
Type                          object
dtype: object

In [11]:
sales_df.head()

,item,item_code,quantity,unit_price,total_sales_amount,date,Type
0,PINT Spotted Cow,3140,64,3.937500,252.00,2021-01-08,Beer
1,DBL RAIL Vodka,3455,37,4.945946,183.00,2021-01-08,Vodka
2,BTL Miller High Life,3122,31,3.250000,100.75,2021-01-08,Beer
3,PINT Stein,3141,29,3.517241,102.00,2021-01-08,Beer
4,SHOT Seagrams VO,3325,26,2.423077,63.00,2021-01-08,Scotch/Whiskey


## Moving Tables to SQLite

After preparing our CSVs, we can move on to import them to the database

**NOTE:** Tables must have already been created (see [data/SQLTableBuild.sql](data/SQLTableBuild.sql))

In [13]:
#save df to sqlite table
sales_df.to_sql('sales', con=engine, if_exists='append')

2021-10-06 19:11:19,116 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sales")
2021-10-06 19:11:19,117 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-06 19:11:19,160 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-06 19:11:19,558 INFO sqlalchemy.engine.Engine INSERT INTO sales ("index", item, item_code, quantity, unit_price, total_sales_amount, date, "Type") VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-10-06 19:11:19,558 INFO sqlalchemy.engine.Engine [generated in 0.36067s] ((0, 'PINT Spotted Cow', 3140, 64, 3.9375, 252.0, '2021-01-08 00:00:00.000000', 'Beer'), (1, 'DBL RAIL Vodka', 3455, 37, 4.9459459459459, 183.0, '2021-01-08 00:00:00.000000', 'Vodka'), (2, 'BTL Miller High Life', 3122, 31, 3.25, 100.75, '2021-01-08 00:00:00.000000', 'Beer'), (3, 'PINT Stein', 3141, 29, 3.5172413793103, 102.0, '2021-01-08 00:00:00.000000', 'Beer'), (4, 'SHOT Seagrams VO', 3325, 26, 2.4230769230769, 63.0, '2021-01-08 00:00:00.000000', 'Scotch/Whiskey'), (5, 'CAN PBR 16oz', 3129, 24, 4.0, 96.

In [17]:
#save df to sqlite table
weather_df.to_sql('weather', con=engine, if_exists="append")

2021-10-06 19:14:48,766 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("weather")
2021-10-06 19:14:48,767 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-06 19:14:48,772 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-06 19:14:48,779 INFO sqlalchemy.engine.Engine INSERT INTO weather ("index", dt, "temp", feels_like, temp_min, temp_max, pressure, humidity, wind_speed, clouds_all, "weather_main_Clear", "weather_main_Clouds", "weather_main_Drizzle", "weather_main_Fog", "weather_main_Haze", "weather_main_Mist", "weather_main_Rain", "weather_main_Smoke", "weather_main_Snow", "weather_main_Thunderstorm") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2021-10-06 19:14:48,780 INFO sqlalchemy.engine.Engine [generated in 0.00475s] ((0, '2019-02-15 00:00:00.000000', 23.368876404494387, 14.429101123595508, -5.1, 41.14, 1018.1011235955056, 76.24719101123596, 11.688089887640446, 50.32022471910113, 0.4101123595505618, 0.2247191011235955, 0.0056179775280898, 0.0056

In [ ]:
#create a new dataframe to hold the merged sales and weather data
new_sales_df = pd.merge(sales_df, weather_df, left_on=sales_df['date'], right_on=weather_df['dt'])
#copying redundant columns
new_sales_df = new_sales_df.drop(columns=['date', 'dt'])
#rename leftmost column
new_sales_df = new_sales_df.rename(columns={'key_0': 'date'})

In [ ]:
new_sales_df